# GSM-ED: Elasto-damage

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
from bmcs_matmod.api import GSM1D_ED
gsm = GSM1D_ED()
display(Markdown(gsm.latex_potentials()))

## Linearly ascending loading

In [ ]:
E_val = 30e+3  # Young's modulus in MPa
K_val = 1e+3  # Bulk modulus in MPa
f_c_val = 15 # Strength in MPa
eta_vp_val = 10e+4  # Viscosity in MPa s
eta_ve_val = 10e+4  # Viscosity in MPa s

# mparams = {gsm.E:10, gsm.c:1, gsm.r:1, gsm.S:10, gsm.eps_0:1}

mparams = dict(E=E_val, K=K_val, f_c=f_c_val, S=0.0001, r=1, c=1,
            eps_0=0.001, eta_ve=eta_ve_val, eta_vp=eta_vp_val)

f_time = 10  # Final time in secondsfinal_time = 1
n_t = 151
t_t = np.linspace(0, f_time, n_t)
eps_max = 0.0025
eps_n_t = np.linspace(0, eps_max, n_t)

response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t_ec, eps_ta_ec, sig_ta_ec, Eps_t_ec, Sig_t_ec, iter_t_ec, lam_t_ec, (d_t_t_ec, d_eps_ta_ec) = response
eps_t_ec = eps_ta_ec[:, 0]
sig_t_ec = sig_ta_ec[:, 0, 0]
# use the stress history to reproduce the response using stress control
response = gsm.get_G_response(sig_t_ec, t_t, **mparams)
t_t_sc, eps_ta_sc, sig_ta_sc, Eps_t_sc, Sig_t_sc, iter_t_sc, lam_t_sc, (d_t_t_sc, d_eps_ta_sc) = response
iter_t = iter_t_sc[:, 0]
eps_t_sc = eps_ta_sc[:, 0, 0]
sig_t_sc = sig_ta_sc[:, 0]
eps_t_ec.shape, sig_t_ec.shape, eps_t_sc.shape, sig_t_sc.shape

In [ ]:
omega_t_ec, z_t_ec = Eps_t_ec[:, 0, :].T
omega_t_sc, z_t_sc = Eps_t_sc[:, 0, :].T
(omega_t_ec.shape, z_t_ec.shape, omega_t_sc.shape, z_t_sc.shape)

In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

# Left: Stress–strain curves (elastic control and stress control)
ax_sig.plot(eps_t_ec, sig_t_ec, color='cadetblue', lw=2, label='Elastic Control')
ax_sig.plot(eps_t_sc, sig_t_sc, color='red', lw=2, label='Stress Control')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.set_ylabel(r'$\sigma$')
ax_sig.legend()

# Right: Damage and hardening profiles
ax_eps.plot(t_t_ec, omega_t_ec, color='darkblue', lw=2, label='Damage')
ax_eps.set_xlabel(r'$t$')
ax_eps.set_ylabel('Damage', color='darkblue')
ax_eps.tick_params(axis='y', labelcolor='darkblue')

# Twin axis for hardening (z)
ax_hard = ax_eps.twinx()
ax_hard.plot(t_t_ec, z_t_ec, color='green', lw=2, label='Hardening')
ax_hard.set_ylabel('Hardening', color='green')
ax_hard.tick_params(axis='y', labelcolor='green')

# Combine legends from both axes
lines1, labels1 = ax_eps.get_legend_handles_labels()
lines2, labels2 = ax_hard.get_legend_handles_labels()
ax_eps.legend(lines1 + lines2, labels1 + labels2, loc='best')

plt.tight_layout()
plt.show()
